# 模型数据处理

## 验证集

一般用dataloader, 然后直接用不同的文件夹处理

## 交叉验证

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


batch_size = 200
learning_rate = 0.01
epochs = 10

train_db = datasets.MNIST(
    '../data', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]))
train_db, val_db = torch.utils.data.random_split(train_db, [50000, 10000])
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_db,
    batch_size=batch_size, shuffle=True)


test_db = datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
test_loader = torch.utils.data.DataLoader(
    test_db,
    batch_size=batch_size, shuffle=True)

print('train:', len(train_db), 'test:', len(test_db))
print('db1:', len(train_db), 'db2:', len(val_db))


train: 50000 test: 10000
db1: 50000 db2: 10000


## 模型过拟合

一般处理多种情况

- 更多的数据
- 减小模型复杂度, regulization正则化
- dropout
- 数据增强
- 提前结束运算

添加L2正则:
```python
device = torch.device('cuda:0')
net = MLP().to(device)
# weight_decay就是lambda 0.01就是l2
optimizer = optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0.01) 
criteon = nn.CrossEntropyLoss().to(device)
```

L1正则比较复杂pytorch并没有携带任何L1代码,需要自己对w进行修正
```python
regularization_loss = 0
for param in model.parameters():
    regularization_loss += torch.sum(torch.abs(param))
classify_loss = criteon(logits, target)
loss = classify_loss + 0.01 * regularization_loss

optimizer.zero_grad()
loss.backward()
optimizer.step()
```

## 动量 momentum

就是在计算梯度的时候添加一个动量让其可以冲过局部最小值. 这里就是增加了一个$\beta$参数让其可以对原始的参数进行增大. 此时我们就得到了一个新的$\beta$函数, 然后用这个函数进行计算.

$$z^{k+1}=\beta{z^k}+\Delta{f(w^k)}$$
$$w^{k+1}=w^k-\alpha{z^{k+1}}$$

pytorch中momentum只需要添加一个参数就可以

```python
optimizer = optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0.01, momentum=args.momentum) 
```

## 梯度递减

第一种方法: 使用ReduceLROnPlateau

这里就是我们的目标是min也就是减小, 我们的目标是监听loss, 如果我们等待了paience次(也就是默认10次)loss没有减小, name我们就减少lr. 减少factor的比重也就是0.1.

```python
# ReduceLROnPlateau(optimier,mode='min',factor=0.1,paience=10,verbose=False,threadhold=0.0001,threshold_mode='rel', cooldown=0,min_lr=0,eps=1e-8)
scheduler = ReduceLROnPlateau(optimizer, 'min')
for epoch in xrange(start,arg.epochs):
    #...
    scheduler.step(loss)
```

第二种方法使用暴力衰减. 直接定义多少次减少
```python
scheduler = StepLR(optimizer, step_size=30,gamma=0.1)
for epoch in range(100):
    #...
    scheduler.step()
    train(...)
    validate(...)
```

## Early stop

略

## dropout

需要注意的是pytorch是drop 概率, 而tensorflow是keep概率, 两个是完全相反的. 巨坑

同时在test的时候一定要eval否则就会随机drop

```python
self.model = nn.Sequential(
    nn.Linear(784, 200),
    nn.Dropout(0.5), # 随机减少50%个神经元链接
    nn.LeakyReLU(inplace=True),
    nn.Linear(200, 200),
    nn.Dropout(0.5), # 随机减少50%个神经元链接
    nn.LeakyReLU(inplace=True),
    nn.Linear(200, 10),
    nn.LeakyReLU(inplace=True),
)
```

## Stochastic Gradient Descent

根据方差进行分布选择. 本身的梯度应该是所有点的梯度平均值, 但是不可能, 所以就是每一个批次进行梯度平均值.